In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from classifier import Data

dataset = Data(
    data_path = "../data/imdb.eval.csv",
    polarities = {
      "negative": 0,
      "positive": 1
    },
    data_label='review',
    target_label='sentiment'
)

In [4]:
from classifier._neural import Encoder

enc = Encoder({
    'model': 'bert-base-uncased',
    'layers': [11]
})

In [5]:
enc.df_encode(dataset.data, col=dataset.data_label, label=dataset.data_path)

In [6]:
import numpy as np
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return MDS(n_components=dim).fit_transform(data)

In [7]:
import torch

embed_col: np.ndarray = torch.stack(dataset.data[enc.col_name].tolist()).numpy()

In [8]:
centroid_records: list = []

In [9]:
import numpy as np

for i in [384, 192, 96, 48, 24, 12, 6, 3]:
    record: dict = {'dim': i}

    dataset.data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=i))

    for label, group in dataset.data.groupby(dataset.target_label):
        record[label] = np.mean(np.stack(group['reduced_embeds'].tolist(), axis=0), axis=0).tolist()

    centroid_records.append(record)

In [10]:
import pandas as pd

pd.DataFrame.from_records(centroid_records).to_csv('./centroid_analysis.csv')